# 1.1 een persoon heeft voor 1 mandaat geen overlappende mandatarissen

We load Javascript libraries to i) retrieve data (bindings) from a bestuurseenheid, ii) validate the document, and iii) display the JSON result.

In [25]:
import { getBestuurseenheidUrl } from "./utils.ts";
import {  getBindingsFromTurtleContent, validateDocument, fetchDocument } from 'npm:@lblod/lib-decision-validation';
import { display } from "https://deno.land/x/display/mod.ts";

const bestuurseenheidUuid = 'd168033a9bac278fa744c425e078eeabd304397f953feaaf5327b4e039aecacb';
const bestuurseenheidUrl = getBestuurseenheidUrl(bestuurseenheidUuid);
const bestuurseenheidBindings = await fetchDocument(bestuurseenheidUrl);

Below, we can find the SPARQL template query and SHACL shape for the 1.1 use case.
The Turtle string is converted into bindings.

In [40]:
const mandataris_1_1_query: string = `
    PREFIX besluit: <http://data.vlaanderen.be/ns/besluit#>
    PREFIX mandaat: <http://data.vlaanderen.be/ns/mandaat#>
    PREFIX org: <http://www.w3.org/ns/org#>
                
    select DISTINCT ($this as ?this) ?value
    where {
        {
            select (count(distinct(?mandataris1)) as ?aantalOverlappendeMandatarissen)
            where {
                ?mandataris1 mandaat:isBestuurlijkeAliasVan $this ;
                                org:holds ?mandaat ;
                                mandaat:start ?startMandataris1 .
                OPTIONAL {
                    ?mandataris1 mandaat:einde ?eindeMandataris1 .
                }
      
      			?mandataris2 mandaat:isBestuurlijkeAliasVan $this ;
                                org:holds ?mandaat ;
                                mandaat:start ?startMandataris2 .
                OPTIONAL {
                    ?mandataris2 mandaat:einde ?eindeMandataris2 .
                }
                FILTER(?mandataris1 != ?mandataris2)

                BIND(strdt(substr(str(?startMandataris1), 1, 10), xsd:date) as ?startMandataris1Date)
                BIND(strdt(substr(str(?eindeMandataris1), 1, 10), xsd:date) as ?eindeMandataris1Date)
                BIND(strdt(substr(str(?startMandataris2), 1, 10), xsd:date) as ?startMandataris2Date)
                BIND(strdt(substr(str(?eindeMandataris2), 1, 10), xsd:date) as ?eindeMandataris2Date)

                BIND((?startMandataris1Date >= ?startMandataris2Date && ?startMandataris1Date <= ?eindeMandataris2Date) AS ?startOverlapt)
                BIND((?eindeMandataris1Date >= ?startMandataris2Date && ?eindeMandataris1Date <= ?eindeMandataris2Date) AS ?eindeOverlapt)

                FILTER(?startOverlapt || ?eindeOverlapt)
            }
            group by ?mandaat
        }
            
        FILTER (?aantalOverlappendeMandatarissen > 0)
        BIND ("Persoon heeft voor 1 mandaat overlappende mandatarissen" as ?value)
    }`;
    
const mandataris_1_1_shape: string = `
@prefix sh: <http://www.w3.org/ns/shacl#> .

<http://example.org/mandataris_1_1_blueprint>
  a sh:NodeShape ;
  sh:targetClass <http://www.w3.org/ns/person#Person> ;
  sh:sparql [
		sh:select """${mandataris_1_1_query}""" ;
        sh:message 'Persoon heeft voor 1 mandaat overlappende/meerdere mandatarissen'
] .`;

const mandataris_1_1_bindings: Bindings[] = await getBindingsFromTurtleContent(mandataris_1_1_shape);

To test the shape, we retrieved a subject from the triple store that is applicable. In this case, we choose a person URI.
Then, the template query from above is applied on this subject.
Next, a Comunica URL is generated to debug the results of this validation.

In [41]:
// Test subject
const subject = 'http://data.lblod.info/id/personen/51ecbbe8e8cdd33ec1c45e69f986b4c6e88dbbcb475747faf69cea6550debb17';
const mandataris_1_1_subject_query = mandataris_1_1_query.replaceAll('$this', `<${subject}>`);

const mandataris_1_1_comunica = `https://query.linkeddatafragments.org/#datasources=${encodeURIComponent(bestuurseenheidUrl)}&query=${encodeURIComponent(mandataris_1_1_subject_query)}`;
console.log(mandataris_1_1_comunica);

https://query.linkeddatafragments.org/#datasources=http%3A%2F%2Flocalhost%3A8890%2Fsparql%2F%3Fdefault-graph-uri%3D%26query%3D%250A%2520%2520%2520%2520construct%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520%253Fs%2520%253Fp%2520%253Fo%2520.%250A%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520where%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520graph%2520%253Chttp%253A%252F%252Fmu.semte.ch%252Fgraphs%252Forganizations%252Fd168033a9bac278fa744c425e078eeabd304397f953feaaf5327b4e039aecacb%252FLoketLB-mandaatGebruiker%253E%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%253Fs%2520%253Fp%2520%253Fo%2520.%250A%2520%2520%2520%2520%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520&query=%0A%20%20%20%20PREFIX%20besluit%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fbesluit%23%3E%0A%20%20%20%20PREFIX%20mandaat%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fmandaat%23%3E%0A%20%20%20%20PREFIX%20org%3A%20%3Chttp%3A%2F%2Fwww.w

The full validation on all persons happens below:

In [42]:
const mandataris_1_1_validationReport = await validateDocument(bestuurseenheidBindings, mandataris_1_1_bindings);
await display({ "application/json": mandataris_1_1_validationReport},{ raw: true });